# Data Processing : Spark SQL, Dataframes, Datasets 
suite (3) ...

### Processsing avec Datasets

In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.115.226:4042
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1659193881311)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Data Processing").
    master("local[8]").
    getOrCreate

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5860b713


Création d'une `case class` pour définir la structure du dataset

In [5]:
case class Person(id: Integer, firstName: String, middleName: String, lastName: String, gender: String, birthDate: String, ssn: String, salary: String)

defined class Person


In [6]:
!head datasets/people-with-header-10m.csv

id,firstName,middleName,lastName,gender,birthDate,ssn,salary
1,Fanchon,Georgeta,Wylma,Female,5/20/2020,537-80-6230,81273.63
2,Sandor,Bordie,Humbert,Male,3/12/2020,640-38-2361,11516.93
3,Quillan,Ara,Hillery,Male,2/3/2020,647-45-5964,37622.42
4,Tallie,Artemis,Urbain,Male,2/13/2020,714-11-1463,63124.83
5,Katharine,Elicia,Muire,Female,2/28/2020,869-39-9071,44189.54
6,Lavinie,Daffie,Xylia,Female,9/9/2020,280-89-8466,48111.47
7,Efrem,Emlen,Beau,Male,4/12/2020,533-59-2497,94346.03
8,Emmaline,Audy,Dorothee,Female,7/14/2020,356-96-4442,73595.17
9,King,Gaston,Jaimie,Male,1/27/2020,239-18-8955,5656.61



In [7]:
val personDS = spark
  .read
  .option("header", "true")
  .option("inferSchema", "true")
  .option("delimiter", ",")
  .csv("datasets/people-with-header-10m.csv")
  .as[Person]

personDS: org.apache.spark.sql.Dataset[Person] = [id: int, firstName: string ... 6 more fields]


In [8]:
personDS

res0: org.apache.spark.sql.Dataset[Person] = [id: int, firstName: string ... 6 more fields]


In [9]:
personDS.show(5)

+---+---------+----------+--------+------+---------+-----------+--------+
| id|firstName|middleName|lastName|gender|birthDate|        ssn|  salary|
+---+---------+----------+--------+------+---------+-----------+--------+
|  1|  Fanchon|  Georgeta|   Wylma|Female|5/20/2020|537-80-6230|81273.63|
|  2|   Sandor|    Bordie| Humbert|  Male|3/12/2020|640-38-2361|11516.93|
|  3|  Quillan|       Ara| Hillery|  Male| 2/3/2020|647-45-5964|37622.42|
|  4|   Tallie|   Artemis|  Urbain|  Male|2/13/2020|714-11-1463|63124.83|
|  5|Katharine|    Elicia|   Muire|Female|2/28/2020|869-39-9071|44189.54|
+---+---------+----------+--------+------+---------+-----------+--------+
only showing top 5 rows



Filtrage de personnes dont le prenom est `Joe`

In [10]:
personDS.filter($"firstName" === "Joe").distinct().show

+---+---------+----------+--------+------+---------+-----------+--------+
| id|firstName|middleName|lastName|gender|birthDate|        ssn|  salary|
+---+---------+----------+--------+------+---------+-----------+--------+
|887|      Joe|    Seumas|     Lon|  Male|1/20/2020|842-80-5339| 33660.7|
| 90|      Joe| Aleksandr|  Goober|  Male| 3/4/1960|236-59-8313|92835.04|
| 69|      Joe|     Dolph|   Grady|  Male|1/24/1980|850-35-5174| 69746.9|
|355|      Joe|    Gaston| Thadeus|  Male| 4/3/1990|479-07-7410| 66034.1|
+---+---------+----------+--------+------+---------+-----------+--------+



In [11]:
// DataFrame Api
// Coompter le nombre de Personne dont le prenom est Joe
println(personDS.filter($"firstName" === "Joe").distinct().count)

4


In [17]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [12]:
// Compter le nombre de Personne dont le prenom est Joe
println(personDS.filter(col("firstName") === "Joe").distinct().count)

4


Filtrage dees personnes agées de 40 ans, qui ont un salaire superieur à 1000$ dont le prénom commence par J et le nom par M.

In [13]:
personDS.printSchema

root
 |-- id: integer (nullable = true)
 |-- firstName: string (nullable = true)
 |-- middleName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- ssn: string (nullable = true)
 |-- salary: double (nullable = true)



In [38]:
val date = "1/20/2020"
date.split("/")(2).toInt > 2019

date: String = 1/20/2020
res29: Boolean = true


In [9]:
personDS
  .filter (split($"birthDate", "/")(2).cast("int") === 2020)
  .show

+---+---------+----------+--------+------+----------+-----------+--------+
| id|firstName|middleName|lastName|gender| birthDate|        ssn|  salary|
+---+---------+----------+--------+------+----------+-----------+--------+
|  1|  Fanchon|  Georgeta|   Wylma|Female| 5/20/2020|537-80-6230|81273.63|
|  2|   Sandor|    Bordie| Humbert|  Male| 3/12/2020|640-38-2361|11516.93|
|  3|  Quillan|       Ara| Hillery|  Male|  2/3/2020|647-45-5964|37622.42|
|  4|   Tallie|   Artemis|  Urbain|  Male| 2/13/2020|714-11-1463|63124.83|
|  5|Katharine|    Elicia|   Muire|Female| 2/28/2020|869-39-9071|44189.54|
|  6|  Lavinie|    Daffie|   Xylia|Female|  9/9/2020|280-89-8466|48111.47|
|  7|    Efrem|     Emlen|    Beau|  Male| 4/12/2020|533-59-2497|94346.03|
|  8| Emmaline|      Audy|Dorothee|Female| 7/14/2020|356-96-4442|73595.17|
|  9|     King|    Gaston|  Jaimie|  Male| 1/27/2020|239-18-8955| 5656.61|
| 11|  Agustin|   Ferrell| Chaunce|  Male| 6/18/2020|172-53-1153|45295.43|
| 12| Clerissa|     Wilie

In [44]:
import org.apache.spark.sql.functions.split

import org.apache.spark.sql.functions.split


In [14]:
import java.util.Calendar
val earliestYear = Calendar.getInstance.get(Calendar.YEAR) - 30
earliestYear

import java.util.Calendar
earliestYear: Int = 1992
res6: Int = 1992


In [61]:
import org.apache.spark.sql.functions._
// definition de la date de naissance des quatergènaire.
import java.util.Calendar
val earliestYear = Calendar.getInstance.get(Calendar.YEAR) - 30

personDS
  .filter(split($"birthDate", "/")(2).cast("int") > earliestYear) // everyone above 30
  .filter($"salary" > 8000.0) // everyone earning more than 80K
  .filter($"firstName".startsWith("J")) // first name starts with J
.show

+---+---------+----------+--------+------+----------+-----------+--------+
| id|firstName|middleName|lastName|gender| birthDate|        ssn|  salary|
+---+---------+----------+--------+------+----------+-----------+--------+
| 57|    Josie|     Mirna| Chelsae|Female| 9/18/2020|258-93-0816|29996.88|
| 60| Julianne|   Celinka| Clarita|Female|  4/6/2020|831-14-9854|79505.56|
| 61|     Jean|      Garv|   Zelig|  Male| 12/1/2020|134-16-0901|92996.35|
| 82|     Jock|      Arie| Herbert|  Male|  1/7/2020|219-82-1860|44868.01|
| 85|   Jamima|     Dasha| Rosanne|Female| 6/14/2020|329-53-9510|31357.74|
|133| Jessamyn|    Ashlie| Vitoria|Female| 12/7/2020|720-94-5366| 45905.3|
|150|   Jammie|   Moselle|   Jandy|Female|12/24/2019|279-44-4635|45918.75|
|151|    Jared|     Dilan|   Doyle|  Male|  1/6/2020|222-95-8092|93273.17|
|167|    Jodie|  Nicolais|  Byrann|  Male|11/15/2020|134-86-4614|76718.58|
|169|     Jena|      Viva|   Ebony|Female| 11/5/2020|639-30-6092|49147.26|
|176|   Jdavie|   Purcell

import org.apache.spark.sql.functions._
import java.util.Calendar
earliestYear: Int = 1991


In [62]:
import org.apache.spark.sql.functions._
// definition de la date de naissance des quatergènaire.
import java.util.Calendar
val earliestYear = Calendar.getInstance.get(Calendar.YEAR) - 30

personDS
  .filter(split($"birthDate", "/")(2).cast("int") > earliestYear) // everyone above 40
  .filter($"salary" > 8000.0) // everyone earning more than 80K
  .filter($"firstName".startsWith("J")) // first name starts with J
  .count()


import org.apache.spark.sql.functions._
import java.util.Calendar
earliestYear: Int = 1991
res49: Long = 45


## Preprocessing avec des données tabulaires

In [63]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Preprocesing column data")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5c43b1af
import spark.implicits._


### Creation d'un DataFrame

In [15]:
val l = List("X")

l: List[String] = List(X)


In [16]:
val df = l.toDF("Vars")

df: org.apache.spark.sql.DataFrame = [Vars: string]


In [17]:
df.printSchema

root
 |-- Vars: string (nullable = true)



In [18]:
df.show

+----+
|Vars|
+----+
|   X|
+----+



In [19]:
val rdd = sc.parallelize(Seq(1, 2, 3, 4))

rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[41] at parallelize at <console>:27


In [20]:
val df = rdd.toDF

df: org.apache.spark.sql.DataFrame = [value: int]


In [21]:
df.show

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
+-----+



Une fois le DataFrame créé, nous pouvons l'utiliser pour comprendre comment utiliser les fonctions. Par exemple, pour obtenir la date actuelle, nous pouvons exécuter `df.select (current_date ()).show()`.

In [16]:
import org.apache.spark.sql.functions.current_date

import org.apache.spark.sql.functions.current_date


In [17]:
df.select(current_date as "Date courant").show

+------------+
|Date courant|
+------------+
|  2022-05-21|
+------------+



In [37]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     ),
                     (5, "Gary", "Smith", 1500.0, 
                      "Australia", "+61 587 656 3280", "789 12 6118")
                     
                    )

employees: List[(Int, String, String, Double, String, String, String)] = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118), (5,Gary,Smith,1500.0,Australia,+61 587 656 3280,789 12 6118))


In [38]:
employees.size

res23: Int = 5


In [39]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )


employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 5 more fields]


In [40]:
employeesDF.printSchema

root
 |-- employee_id: integer (nullable = false)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: double (nullable = false)
 |-- nationality: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- ssn: string (nullable = true)



In [41]:
employeesDF.createTempView("Employee")

In [44]:
spark.sql("select nationality, count(1) as count from Employee group by nationality").show

+--------------+-----+
|   nationality|count|
+--------------+-----+
|         India|    1|
|united KINGDOM|    1|
| united states|    1|
|     AUSTRALIA|    1|
|     Australia|    1|
+--------------+-----+



In [45]:
spark.sql("select concat(first_name,' - ',last_name) as name from Employee").show

+-------------+
|         name|
+-------------+
|Scott - Tiger|
| Henry - Ford|
|Nick - Junior|
| Bill - Gomes|
| Gary - Smith|
+-------------+



In [28]:
employeesDF.show(false)

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|nationality   |phone_number    |ssn        |
+-----------+----------+---------+------+--------------+----------------+-----------+
|1          |Scott     |Tiger    |1000.0|united states |+1 123 456 7890 |123 45 6789|
|2          |Henry     |Ford     |1250.0|India         |+91 234 567 8901|456 78 9123|
|3          |Nick      |Junior   |750.0 |united KINGDOM|+44 111 111 1111|222 33 4444|
|4          |Bill      |Gomes    |1500.0|AUSTRALIA     |+61 987 654 3210|789 12 6118|
|5          |Gary      |Smith    |1500.0|Australia     |+61 587 656 3280|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [29]:
employeesDF.
    select("first_name", "last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
|      Gary|    Smith|
+----------+---------+



### Utilisation des fonctions Spark

In [85]:
// Pour l'utilisation de $ dans les functions à la place de col
import spark.implicits._

import spark.implicits._


In [30]:
employeesDF.
    select($"first_name", $"last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
|      Gary|    Smith|
+----------+---------+



In [87]:
// Pour l'utilisation de col
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [88]:
// Utilisation de col 

employeesDF.
    select(col("first_name"), $"last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
|      Gary|    Smith|
+----------+---------+



In [89]:
// Utilisation du nom des colonnes.
employeesDF.
    select("first_name", "last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
|      Gary|    Smith|
+----------+---------+



### groupBy colonne

In [33]:
employeesDF.
    groupBy("nationality").
    count.
    show

+--------------+-----+
|   nationality|count|
+--------------+-----+
|         India|    1|
|united KINGDOM|    1|
| united states|    1|
|     AUSTRALIA|    1|
|     Australia|    1|
+--------------+-----+



In [34]:
import org.apache.spark.sql.functions.upper

import org.apache.spark.sql.functions.upper


In [34]:
employeesDF.
    groupBy(upper($"nationality")).
    count.
    show

+------------------+-----+
|upper(nationality)|count|
+------------------+-----+
|    UNITED KINGDOM|    1|
|             INDIA|    1|
|         AUSTRALIA|    2|
|     UNITED STATES|    1|
+------------------+-----+



In [ ]:
df.select
.filter
.where
.groupBy
.orderBy
.count
.agg
.join

### orderBy colonne

In [24]:
employeesDF.
    orderBy(asc("employee_id")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          5|      Gary|    Smith|1500.0|     Australia|+61 587 656 3280|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



Cependant, si nous voulons appliquer une transformation à l'aide de fonctions, passer des noms de colonnes sous forme de chaînes à certaines des fonctions ne suffira pas. Nous devons les passer comme type de colonne.

In [37]:
import org.apache.spark.sql.functions.desc

import org.apache.spark.sql.functions.desc


In [96]:
employeesDF.
    orderBy(desc("first_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          5|      Gary|    Smith|1500.0|     Australia|+61 587 656 3280|789 12 6118|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [97]:
employeesDF.
    orderBy(col("employee_id").desc).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          5|      Gary|    Smith|1500.0|     Australia|+61 587 656 3280|789 12 6118|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [40]:
//Error
employeesDF.
    select(lower("first_name")).
    show

<console>: 49: error: type mismatch;

In [43]:
import org.apache.spark.sql.functions.upper

import org.apache.spark.sql.functions.upper


In [98]:
// version correcte
employeesDF.
    select(upper(col("first_name"))).
    show

+-----------------+
|upper(first_name)|
+-----------------+
|            SCOTT|
|            HENRY|
|             NICK|
|             BILL|
|             GARY|
+-----------------+



In [99]:
// version alternative
employeesDF.
    select(upper($"first_name")).
    show

+-----------------+
|upper(first_name)|
+-----------------+
|            SCOTT|
|            HENRY|
|             NICK|
|             BILL|
|             GARY|
+-----------------+



In [100]:
// dans le cas de groupBy
employeesDF.
    groupBy(upper($"nationality")).
    count.
    show

+------------------+-----+
|upper(nationality)|count|
+------------------+-----+
|    UNITED KINGDOM|    1|
|             INDIA|    1|
|         AUSTRALIA|    2|
|     UNITED STATES|    1|
+------------------+-----+



In [101]:
// Dans le cas de ordeBy
employeesDF.
    orderBy(upper($"nationality")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          5|      Gary|    Smith|1500.0|     Australia|+61 587 656 3280|789 12 6118|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [48]:
// version alternative
employeesDF.
    orderBy(upper(employeesDF("nationality"))).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [25]:
employeesDF("first_name")

res15: org.apache.spark.sql.Column = first_name


### Concatenation

In [104]:
import org.apache.spark.sql.functions.{concat,lit}

import org.apache.spark.sql.functions.{concat, lit}


Versions incorrectes de concatenation

In [35]:
employeesDF.
    select(concat($"first_name", ", ", $"last_name")).
    show()

<console>: 31: error: type mismatch;

In [106]:
// Deme
employeesDF.
    select(concat(col("first_name"), ", ", col("last_name"))).
    show

<console>: 100: error: type mismatch;

In [107]:
employeesDF.
    select(concat(employeesDF("first_name"), ", ", employeesDF("last_name"))).
    show

<console>: 99: error: type mismatch;

In [53]:
employeesDF.
    select(concat("first_name", ", ", "last_name")).
    show


<console>: 51: error: type mismatch;

versions correctes

In [54]:
import org.apache.spark.sql.functions.{concat, col, lit}

import org.apache.spark.sql.functions.{concat, col, lit}


In [36]:
employeesDF.
    select(concat(col("first_name"), lit(" - "), col("last_name")) as "Full Name").
    withColumn("Status", lit("yes")).
    show

+-------------+------+
|    Full Name|Status|
+-------------+------+
|Scott - Tiger|   yes|
| Henry - Ford|   yes|
|Nick - Junior|   yes|
| Bill - Gomes|   yes|
| Gary - Smith|   yes|
+-------------+------+



In [56]:
employeesDF.
    select(concat($"first_name", lit(", "), employeesDF("last_name"))).
    show

+---------------------------------+
|concat(first_name, , , last_name)|
+---------------------------------+
|                     Scott, Tiger|
|                      Henry, Ford|
|                     Nick, Junior|
|                      Bill, Gomes|
+---------------------------------+



### Manipulation de chaines de caractères

In [57]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )


employees: List[(Int, String, String, Double, String, String, String)] = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


In [58]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 5 more fields]


In [59]:
import org.apache.spark.sql.functions.{col, upper, lower, initcap, length}

import org.apache.spark.sql.functions.{col, upper, lower, initcap, length}


In [60]:
employeesDF.
    select("employee_id", "nationality").
    withColumn("nationality_upper", upper(col("nationality"))).
    withColumn("nationality_lower", lower($"nationality")).
    withColumn("nationality_initcap", initcap(employeesDF("nationality"))).
    withColumn("nationality_length", length(col("nationality"))).
    show

+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|employee_id|   nationality|nationality_upper|nationality_lower|nationality_initcap|nationality_length|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|          1| united states|    UNITED STATES|    united states|      United States|                13|
|          2|         India|            INDIA|            india|              India|                 5|
|          3|united KINGDOM|   UNITED KINGDOM|   united kingdom|     United Kingdom|                14|
|          4|     AUSTRALIA|        AUSTRALIA|        australia|          Australia|                 9|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+



**Substring**  
Extraction de sous chaine avec la fonction substring. Cette fonction prend en argument la colonne et la position initiale et le nombre de charactère à extraire.

In [114]:
import org.apache.spark.sql.functions.substring

import org.apache.spark.sql.functions.substring


In [115]:
import spark.implicits._

import spark.implicits._


In [63]:
employeesDF.
    select("employee_id", "phone_number", "ssn").
    withColumn("phone_last4", substring($"phone_number", -4, 4).cast("int")).
    withColumn("ssn_last4", substring($"ssn", 8, 4).cast("int")).
    show

+-----------+----------------+-----------+-----------+---------+
|employee_id|    phone_number|        ssn|phone_last4|ssn_last4|
+-----------+----------------+-----------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|       3210|     6118|
+-----------+----------------+-----------+-----------+---------+



**Splitting**  
Il permet de découper une chaine à partir d'un délimiteur (exemple de delimiteur ' ', ',', ';' etc.).

In [116]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )

employees: List[(Int, String, String, Double, String, String, String)] = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


In [65]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 5 more fields]


In [66]:
import org.apache.spark.sql.functions.split

import org.apache.spark.sql.functions.split


In [67]:
import spark.implicits._

import spark.implicits._


In [68]:
employeesDF.
    select("employee_id", "phone_number", "ssn").
    withColumn("area_code", split($"phone_number", " ")(1).cast("int")).
    withColumn("phone_last4", split($"phone_number", " ")(3).cast("int")).
    withColumn("ssn_last4", split($"ssn", " ")(2).cast("int")).
    show

+-----------+----------------+-----------+---------+-----------+---------+
|employee_id|    phone_number|        ssn|area_code|phone_last4|ssn_last4|
+-----------+----------------+-----------+---------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|      123|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|      234|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|      111|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|      987|       3210|     6118|
+-----------+----------------+-----------+---------+-----------+---------+



In [69]:
employeesDF.
    select($"employee_id", $"phone_number", $"ssn", 
           split($"phone_number", " ")(1).cast("int").alias("area_code"),
           split($"phone_number", " ")(3).cast("int").alias("phone_last"),
           split($"ssn", " ")(2).cast("int").alias("ssn_last4")
          ).
    show

+-----------+----------------+-----------+---------+----------+---------+
|employee_id|    phone_number|        ssn|area_code|phone_last|ssn_last4|
+-----------+----------------+-----------+---------+----------+---------+
|          1| +1 123 456 7890|123 45 6789|      123|      7890|     6789|
|          2|+91 234 567 8901|456 78 9123|      234|      8901|     9123|
|          3|+44 111 111 1111|222 33 4444|      111|      1111|     4444|
|          4|+61 987 654 3210|789 12 6118|      987|      3210|     6118|
+-----------+----------------+-----------+---------+----------+---------+



`withColumn` permet la creation d'une nouvelle colonne dans le dataframe

In [70]:
import org.apache.spark.sql.functions.concat
import spark.implicits._

import org.apache.spark.sql.functions.concat
import spark.implicits._


In [71]:
employeesDF.
    withColumn("full_name", concat($"first_name", $"last_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn| full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|ScottTiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| HenryFord|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|NickJunior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| BillGomes|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+



In [72]:
import org.apache.spark.sql.functions.{concat, lit}

import org.apache.spark.sql.functions.{concat, lit}


In [73]:
employeesDF.
    withColumn("full_name", concat($"first_name", lit(", "), $"last_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|   full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|Scott, Tiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| Henry, Ford|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|Nick, Junior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| Bill, Gomes|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+



### Padding  
Permet le remplissage d'une pattern avec une caractère par défaut.

In [74]:
import org.apache.spark.sql.functions.{lit, lpad}

import org.apache.spark.sql.functions.{lit, lpad}


In [75]:
val l = List("X")

l: List[String] = List(X)


In [76]:
val df = l.toDF("dummy")

df: org.apache.spark.sql.DataFrame = [dummy: string]


In [77]:
df.select(lpad(lit("Hello"), 10, "-").alias("dummy")).show

+----------+
|     dummy|
+----------+
|-----Hello|
+----------+



Utilisez les fonctions de pad pour convertir chacun des champs en longueur fixe et concaténer. Voici les détails de chacun des champs.

* La longueur de employee_id doit être de 5 caractères et doit être complétée par zéro.
* La longueur de first_name et last_name doit être de 10 caractères et doit être complétée par - sur le côté droit.
* La longueur du salaire doit être de 10 caractères et doit être complétée par zéro.
*  La longueur de la nationalité doit être de 15 caractères et doit être complétée par - sur le côté droit.
* La longueur du phone_number doit être de 17 caractères et doit être complétée par - sur le côté droit.
* La longueur du ssn peut être laissée telle quelle. C'est 11 caractères.

In [78]:
import org.apache.spark.sql.functions.{lpad, rpad, concat}

import org.apache.spark.sql.functions.{lpad, rpad, concat}


In [79]:
import spark.implicits._

import spark.implicits._


In [80]:
val empFixedDF = employeesDF.select(
    concat(
        lpad($"employee_id", 5, "0"),
        rpad($"first_name", 10, "-"),
        rpad($"last_name", 10, "-"),
        lpad($"salary", 10, "0"),
        rpad($"nationality", 15, "-"),
        rpad($"phone_number", 17, "-"),
        $"ssn"
    ).alias("employee")
)

empFixedDF: org.apache.spark.sql.DataFrame = [employee: string]


In [81]:
empFixedDF.show

+--------------------+
|            employee|
+--------------------+
|00001Scott-----Ti...|
|00002Henry-----Fo...|
|00003Nick------Ju...|
|00004Bill------Go...|
+--------------------+



In [82]:
empFixedDF.show(false)

+------------------------------------------------------------------------------+
|employee                                                                      |
+------------------------------------------------------------------------------+
|00001Scott-----Tiger-----00001000.0united states--+1 123 456 7890--123 45 6789|
|00002Henry-----Ford------00001250.0India----------+91 234 567 8901-456 78 9123|
|00003Nick------Junior----00000750.0united KINGDOM-+44 111 111 1111-222 33 4444|
|00004Bill------Gomes-----00001500.0AUSTRALIA------+61 987 654 3210-789 12 6118|
+------------------------------------------------------------------------------+



### Trimming  
Il permet de supprimer les espaces vides ou une caractère au début ou à la fin d'une chaine.

In [83]:
val l = List("   Hello.    ")

l: List[String] = List("   Hello.    ")


In [84]:
val df = l.toDF("dummy")

df: org.apache.spark.sql.DataFrame = [dummy: string]


In [85]:
import org.apache.spark.sql.functions.{col, ltrim, rtrim, trim}

import org.apache.spark.sql.functions.{col, ltrim, rtrim, trim}


In [86]:
df.withColumn("ltrim", ltrim(col("dummy"))).
    withColumn("rtrim", rtrim(rtrim(col("dummy")), ".")).
    withColumn("trim", trim(trim(col("dummy")), ".")).
    show()

+-------------+----------+--------+-----+
|        dummy|     ltrim|   rtrim| trim|
+-------------+----------+--------+-----+
|   Hello.    |Hello.    |   Hello|Hello|
+-------------+----------+--------+-----+



## Date et Time

Nous allons avoir un aperçu de la date et de l'heure en utilisant les fonctions disponibles.  
Nous pouvons utiliser current_date pour obtenir la date du serveur du jour.  
La date sera retournée au format `yyyy-MM-dd`.    
Nous pouvons utiliser current_timestamp pour obtenir l'heure actuelle du serveur.  
Timestamp permet de renvoyé au format `yyyy-MM-dd HH:mm:ss.SSS`.  
Les heures seront par défaut au format 24 heures.  


In [87]:
val l = List("X")
val df = l.toDF("dummy")

l: List[String] = List(X)
df: org.apache.spark.sql.DataFrame = [dummy: string]


In [88]:
import org.apache.spark.sql.functions.{current_date, current_timestamp}

import org.apache.spark.sql.functions.{current_date, current_timestamp}


In [89]:
df.select(current_date.alias("current_date")).show

+------------+
|current_date|
+------------+
|  2021-06-30|
+------------+



In [90]:
df.select(current_timestamp.alias("current_time")).show(false)

+--------------------------+
|current_time              |
+--------------------------+
|2021-06-30 22:17:03.689213|
+--------------------------+



In [91]:
val datetimes = List(("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                    )

datetimes: List[(String, String)] = List((2014-02-28,2014-02-28 10:00:00.123), (2016-02-29,2016-02-29 08:08:08.999), (2017-10-31,2017-12-31 11:59:59.123), (2019-11-30,2019-08-31 00:00:00.000))


In [92]:
val datetimesDF = datetimes.toDF("date", "time")

datetimesDF: org.apache.spark.sql.DataFrame = [date: string, time: string]


In [93]:
datetimesDF.show(false)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+




## Date and Time - Arithmetic

Effectuons des opérations arithmétique sur la date et l'heure à l'aide des fonctions appropriées.

- Ajout de jours à une date ou un timestamp - date_add
- Soustraire des jours d'une date ou d'un timestamp - date_sub
- Obtenir la différence entre 2 dates ou timestamp - datediff
- Obtenir le nombre de mois entre 2 dates ou timestamp - months_between
- Ajout de mois à une date ou un timestamp - add_months
- Obtenir le jour suivant à partir d'une date donnée - next_day
- Toutes ces fonctions sont explicites. Nous pouvons les appliquer sur une date ou un timestamp standard. Toutes les fonctions renvoient une date même lorsqu'elles sont appliquées à un champ de timestamp.


### A faire
    
Effectuons quelques opérations arithmétique sur les dates.

- Obtenez d'abord à l'aide sur chaque fonction et comprenez quels sont les arguments qui doivent être passés.
- Créez un Dataframe du nom de datetimesDF avec les colonnes date et time.



In [94]:
val datetimes = List(("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                    )

datetimes: List[(String, String)] = List((2014-02-28,2014-02-28 10:00:00.123), (2016-02-29,2016-02-29 08:08:08.999), (2017-10-31,2017-12-31 11:59:59.123), (2019-11-30,2019-08-31 00:00:00.000))


In [95]:
val datetimesDF = datetimes.toDF("date", "time")

datetimesDF: org.apache.spark.sql.DataFrame = [date: string, time: string]


In [96]:
datetimesDF.show(false)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+




- Ajouter 10 jours aux valeurs de la colonne date et time.
- Soustraire 10 jours aux valeurs de la colonne date et time.
- calculer la différence entre `current_date` et les valeurs de la colonne date ainsi que `current_timestamp` et les valeurs de la colonne time.
- calculer le nombre de mois entre l`current_date` et les valeurs de la colonne date ainsi que `current_timestamp`et les valeurs de la colonne time.
- Ajoutez 3 mois sur les valeurs de la colonnes de date et time.

In [97]:
import org.apache.spark.sql.functions.{date_add, date_sub}

import org.apache.spark.sql.functions.{date_add, date_sub}


In [98]:
datetimesDF.
    withColumn("date_add_date", date_add($"date", 10)).
    withColumn("date_add_time", date_add($"time", 10)).
    withColumn("date_sub_date", date_sub($"date", 10)).
    withColumn("date_sub_time", date_sub($"time", 10)).
    show(false)

+----------+-----------------------+-------------+-------------+-------------+-------------+
|date      |time                   |date_add_date|date_add_time|date_sub_date|date_sub_time|
+----------+-----------------------+-------------+-------------+-------------+-------------+
|2014-02-28|2014-02-28 10:00:00.123|2014-03-10   |2014-03-10   |2014-02-18   |2014-02-18   |
|2016-02-29|2016-02-29 08:08:08.999|2016-03-10   |2016-03-10   |2016-02-19   |2016-02-19   |
|2017-10-31|2017-12-31 11:59:59.123|2017-11-10   |2018-01-10   |2017-10-21   |2017-12-21   |
|2019-11-30|2019-08-31 00:00:00.000|2019-12-10   |2019-09-10   |2019-11-20   |2019-08-21   |
+----------+-----------------------+-------------+-------------+-------------+-------------+



In [99]:
import org.apache.spark.sql.functions.{current_date, current_timestamp, datediff}

import org.apache.spark.sql.functions.{current_date, current_timestamp, datediff}


In [100]:
datetimesDF.
    withColumn("datediff_date", datediff(current_date, $"date")).
    withColumn("datediff_time", datediff(current_timestamp, $"time")).
    show(false)

+----------+-----------------------+-------------+-------------+
|date      |time                   |datediff_date|datediff_time|
+----------+-----------------------+-------------+-------------+
|2014-02-28|2014-02-28 10:00:00.123|2679         |2679         |
|2016-02-29|2016-02-29 08:08:08.999|1948         |1948         |
|2017-10-31|2017-12-31 11:59:59.123|1338         |1277         |
|2019-11-30|2019-08-31 00:00:00.000|578          |669          |
+----------+-----------------------+-------------+-------------+



In [101]:
import org.apache.spark.sql.functions.{months_between, add_months, round}

import org.apache.spark.sql.functions.{months_between, add_months, round}


In [102]:
datetimesDF.
    withColumn("months_between_date", round(months_between(current_date, $"date"), 2)).
    withColumn("months_between_time", round(months_between(current_timestamp, $"time"), 2)).  
    withColumn("add_months_date", add_months($"date", 3)).
    withColumn("add_months_time", add_months($"time", 3)).
    show(false)

+----------+-----------------------+-------------------+-------------------+---------------+---------------+
|date      |time                   |months_between_date|months_between_time|add_months_date|add_months_time|
+----------+-----------------------+-------------------+-------------------+---------------+---------------+
|2014-02-28|2014-02-28 10:00:00.123|88.0               |88.0               |2014-05-28     |2014-05-28     |
|2016-02-29|2016-02-29 08:08:08.999|64.0               |64.0               |2016-05-29     |2016-05-29     |
|2017-10-31|2017-12-31 11:59:59.123|44.0               |42.0               |2018-01-31     |2018-03-31     |
|2019-11-30|2019-08-31 00:00:00.000|19.0               |22.0               |2020-02-29     |2019-11-30     |
+----------+-----------------------+-------------------+-------------------+---------------+---------------+



## Date et heure - trunc et date_trunc

Dans le contexte du Data Warehousing, nous exécutons assez souvent des rapports à ce jour tels que des rapports hebdomadaires, mensuels, annuels, etc.

- Nous pouvons utiliser `trunc` ou `date_trunc` pour obtenir la date de début de la semaine, du mois, de l'année en cours, etc. en lui passant date ou timestamp.
- Nous pouvons utiliser `trunc` pour obtenir la date de début du mois ou de l'année en lui passant date ou timestamp  - par exemple, trunc(current_date(), "MM") donnera le premier du mois en cours.
- Nous pouvons utiliser date_trunc pour obtenir la date de début du mois ou de l'année ainsi que l'heure de début du jour ou de l'heure en lui passant l'horodatage.
- Obtenir la date de début en fonction du mois - date_trunc("MM", current_timestamp())
- Obtenir l'heure de début en fonction du jour - date_trunc("DAY", current_timestamp())

### Tâches

Exécutons quelques tâches pour comprendre en détail trunc et date_trunc.

Créez un Dataframe par nom datetimesDF avec les colonnes date et heure.



In [103]:
val datetimes = List(("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                    )

datetimes: List[(String, String)] = List((2014-02-28,2014-02-28 10:00:00.123), (2016-02-29,2016-02-29 08:08:08.999), (2017-10-31,2017-12-31 11:59:59.123), (2019-11-30,2019-08-31 00:00:00.000))


In [104]:
val datetimesDF = datetimes.toDF("date", "time")

datetimesDF: org.apache.spark.sql.DataFrame = [date: string, time: string]


In [105]:
datetimesDF.show(truncate=false)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



In [106]:
import org.apache.spark.sql.functions.trunc

import org.apache.spark.sql.functions.trunc


In [107]:
import spark.implicits._

import spark.implicits._


In [108]:
datetimesDF.
    withColumn("date_trunc", trunc($"date", "MM")).
    withColumn("time_trunc", trunc($"time", "yyyy")).
    show(false)

+----------+-----------------------+----------+----------+
|date      |time                   |date_trunc|time_trunc|
+----------+-----------------------+----------+----------+
|2014-02-28|2014-02-28 10:00:00.123|2014-02-01|2014-01-01|
|2016-02-29|2016-02-29 08:08:08.999|2016-02-01|2016-01-01|
|2017-10-31|2017-12-31 11:59:59.123|2017-10-01|2017-01-01|
|2019-11-30|2019-08-31 00:00:00.000|2019-11-01|2019-01-01|
+----------+-----------------------+----------+----------+



In [109]:
import org.apache.spark.sql.functions.date_trunc

import org.apache.spark.sql.functions.date_trunc


In [110]:
datetimesDF.
    withColumn("date_dt", date_trunc("HOUR", $"date")).
    withColumn("time_dt", date_trunc("HOUR", $"time")).
    show(false)

+----------+-----------------------+-------------------+-------------------+
|date      |time                   |date_dt            |time_dt            |
+----------+-----------------------+-------------------+-------------------+
|2014-02-28|2014-02-28 10:00:00.123|2014-02-28 00:00:00|2014-02-28 10:00:00|
|2016-02-29|2016-02-29 08:08:08.999|2016-02-29 00:00:00|2016-02-29 08:00:00|
|2017-10-31|2017-12-31 11:59:59.123|2017-10-31 00:00:00|2017-12-31 11:00:00|
|2019-11-30|2019-08-31 00:00:00.000|2019-11-30 00:00:00|2019-08-31 00:00:00|
+----------+-----------------------+-------------------+-------------------+



## Date et heure - Extraction d'informations

Essayons de comprendre comment extraire des informations à partir de la dates ou de l'heures à l'aide de fonctions.

Nous pouvons utiliser date_format pour extraire les informations requises dans un format souhaité à partir de  date ou timestamp.

Il existe également des fonctions spécifiques pour extraire l'année, le mois, le jour d'une semaine, un jour d'un mois, un jour d'une année etc.

### Tâches

Exécutons quelques tâches pour extraire les informations dont nous avons besoin de la date ou le timestamp.

Créez un Dataframe par nom datetimesDF avec les colonnes date et heure.



In [111]:
val datetimes = List(("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                    )

datetimes: List[(String, String)] = List((2014-02-28,2014-02-28 10:00:00.123), (2016-02-29,2016-02-29 08:08:08.999), (2017-10-31,2017-12-31 11:59:59.123), (2019-11-30,2019-08-31 00:00:00.000))


In [112]:
val datetimesDF = datetimes.toDF("date", "time")

datetimesDF: org.apache.spark.sql.DataFrame = [date: string, time: string]
